In [6]:
import numpy as np
from scipy.integrate import solve_ivp
from scipy.interpolate import interp1d
from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure 
from bokeh.layouts import row, column
from ipywidgets import interact
output_notebook()

Loading BokehJS ...

In [7]:
# Defining parameters
k1 = 0.2  # 1/s
k2 = 0.6  # 1/s
vmax = 25.0  # m/s
tdes = 1.0  # s
hbuf = 1.06
alpha_p = 0.4  # m/s^2
vp = 15.0  # m/s
va = 23.0  # m/s
tp = 10.0  # s
ta = 120.0  # s
d = 7.5 # m
ts1 = 20.0  # s
init_sep = (d + tdes * vmax) * hbuf
timelist = []
poslist = []
tmax = 400
num_cars = 200
abs_car = 50
timesteps = 100
car_pos = np.zeros(timesteps)
init_car_pos = np.linspace(num_cars * init_sep, 0, num_cars) 
car_vel = np.zeros(timesteps)
init_car_vel = np.empty(num_cars)
init_car_vel.fill(vmax)

In [8]:
# Setup plots
# Figure 1
p1 = figure(plot_width=500, plot_height=500, title='Time-Space Diagram with JAD',
           x_axis_label='Time', y_axis_label='Position')
r11 = p1.multi_line(timelist, poslist)

In [9]:
# def RunJAD():
def LeadingCar(t, y):
    x, v = y
    dydt = [v, a0]
    return dydt
def AbsorbingCar(t, y):
    x, v = y
    dydt = [v, a1]
    return dydt
def FollowingCar(t, y):
    x, v = y
    dydt = [v, -(k2 + k1 * tdes) * v - k1 * x + vfront(t) * k2 + xfront(t) * k1 - k1 * d]
    return dydt
def SolveCarFollow(a0, a1, tl1, tl2, ta1, ta2):
    y0 = [init_car_pos[0], init_car_vel[0]]
    sol = solve_ivp(LeadingCar, (tl1, tl2), y0, method='RK45', t_eval=timepoints1)
    global car_pos, car_vel
    car_pos = np.copy(sol.y[0, :])
    car_vel = np.copy(sol.y[1, :])
    np.clip(car_vel, 0, vmax, out=car_vel)
    init_car_pos[0] = car_pos[-1]
    init_car_vel[0] = car_vel[-1]
    timelist.append(sol.t)
    poslist.append(car_pos)
    for i in range(1, abs_car):
        y0 = [init_car_pos[i], init_car_vel[i]]
        global xfront, vfront
        xfront = interp1d(timepoints1, car_pos, bounds_error=False, fill_value='extrapolate')
        vfront = interp1d(timepoints1, car_vel, bounds_error=False, fill_value='extrapolate')
        sol = solve_ivp(FollowingCar, (tl1, tl2), y0, method='RK45', t_eval=timepoints1)
        car_pos = np.copy(sol.y[0, :])
        car_vel = np.copy(sol.y[1, :])
        np.clip(car_vel, 0, vmax, out=car_vel)
        init_car_pos[i] = car_pos[-1]
        init_car_vel[i] = car_vel[-1]
        if np.mod(i, 5) == 0:
            timelist.append(sol.t)
            poslist.append(car_pos)
    y0 = [init_car_pos[abs_car], init_car_vel[abs_car]]
    sol = solve_ivp(LeadingCar, (ta1, ta2), y0, method='RK45', t_eval=timepoints2)
    car_pos = np.copy(sol.y[0, :])
    car_vel = np.copy(sol.y[1, :])
    np.clip(car_vel, 0, vmax, out=car_vel)
    init_car_pos[abs_car] = car_pos[-1]
    init_car_vel[abs_car] = car_vel[-1]
    timelist.append(sol.t)
    poslist.append(car_pos)
    for i in range(abs_car + 1, num_cars):
        y0 = [init_car_pos[i], init_car_vel[i]]
        xfront = interp1d(timepoints2, car_pos, bounds_error=False, fill_value='extrapolate')
        vfront = interp1d(timepoints2, car_vel, bounds_error=False, fill_value='extrapolate')
        sol = solve_ivp(FollowingCar, (ta1, ta2), y0, method='RK45', t_eval=timepoints2)
        car_pos = np.copy(sol.y[0, :])
        car_vel = np.copy(sol.y[1, :])
        np.clip(car_vel, 0, vmax, out=car_vel)
        init_car_pos[i] = car_pos[-1]
        init_car_vel[i] = car_vel[-1]
        if np.mod(i, 5) == 0:
            timelist.append(sol.t)
            poslist.append(car_pos)

In [10]:
# Constant velocity phase 1
a0 = 0
a1 = 0
tl1 = 0
tl2 = ts1
ta1 = 0
ta2 = ts1
timepoints1 = np.linspace(tl1, tl2, timesteps)
timepoints2 = np.linspace(ta1, ta2, timesteps)
xfront = interp1d(timepoints1, car_pos)
vfront = interp1d(timepoints1, car_vel)
SolveCarFollow(a0, a1, tl1, tl2, ta1, ta2)

# Constant deceleration phase
a0 = -alpha_p
a1 = -alpha_p
tl3 = tl2 + (vp - vmax) / a0
ta3 = ta2 + (va - vmax) / a1
timepoints1 = np.linspace(tl2, tl3, timesteps)
timepoints2 = np.linspace(ta2, ta3, timesteps)
SolveCarFollow(a0, a1, tl2, tl3, ta2, ta3)

# Constant velocity phase 2
a0 = 0
a1 = 0
tl4 = tl3 + tp
ta4 = ta3 + ta
timepoints1 = np.linspace(tl3, tl4, timesteps)
timepoints2 = np.linspace(ta3, ta4, timesteps)
SolveCarFollow(a0, a1, tl3, tl4, ta3, ta4)

# Constant acceleration phase
a0 = alpha_p
a1 = alpha_p
tl5 = tl4 + (vmax - vp) / a0
ta5 = ta4 + (vmax - va) / a1
timepoints1 = np.linspace(tl4, tl5, timesteps)
timepoints2 = np.linspace(ta4, ta5, timesteps)
SolveCarFollow(a0, a1, tl4, tl5, ta4, ta5)

# Constant velocity phase 3
a0 = 0
a1 = 0
tl6 = tmax
ta6 = tmax
timepoints1 = np.linspace(tl5, tl6, timesteps)
timepoints2 = np.linspace(ta5, ta6, timesteps)
SolveCarFollow(a0, a1, tl5, tl6, ta5, ta6)

In [11]:
r11.data_source.data['x'] = timelist
r11.data_source.data['y'] = poslist
push_notebook(handle=target1)

NameError: name 'target1' is not defined

In [12]:
target1 = show(p1, notebook_handle=True)